## Multiclass Case

In [1]:
# libraries
import numpy as np
import cupy as cp

In [25]:
# other test variables, which are self parameters
# classes schould be given as int from 0 to k
n_total_ftr = 3
target_values = [0,1]
mu = cp.ones((n_total_ftr, len(target_values))) * 0
sigma = cp.ones((n_total_ftr, len(target_values))) * 1
penalty_s = 0.01
penalty_r = 0.01
epochs = 1
lr_mu = 1
lr_sigma = 1
n_mc_samples = 2 #10000 # monte carlo samples

# create 3d array with all r for current observation 
# r^cl_j = r[l, j, c]
#needs to be specified only if we got right model
#r = np.random.randn(monte_carlo, n_total_ftr, len(target_values))

# maybe set param amount_classes from target_values

In [26]:
def __softmax( x, y):

        """
        Update the distribution parameters mu and sigma by optimizing them in terms of the (log) likelihood.
        Here we assume a multinominal distributed target variable. We use a Multinominal model as our base model.
        Funciton with cupy functions.

        :param x: (np.ndarray) Batch of observations (numeric values only, consider normalizing data for better results)
        :param y: (np.ndarray) Batch of labels: type integer e.g. 0,1,2,3,4 etc.
        """
        global sigma
        global mu
        global epochs
        global lr_mu
        global lr_sigma
        global n_mc_samples
        if len(x.shape) != 2:
            x = x.reshape(1,len(x))
    
        observed_classes = np.unique(y)

        for obs_class in observed_classes:
            observations_index = np.where(y == obs_class)[0]
            x_obs = cp.array(x[observations_index])
            print("x:")
            print(x_obs)
            n_obs = len(x_obs)
            print("y: {}".format(obs_class))
            #print("obs_class: {}, n obs: {}".format(obs_class, n_obs))

            for epoch in range(epochs):
                    
                    # Iterative update of mu and sigma
                    try:
                        # o number of obs, l number of samples, j features,
                        # c classes
                        
                        # r shape: oxlxjxc
                        r = cp.random.randn(n_obs, n_mc_samples,
                                            n_total_ftr,
                                            len(target_values))

                        r[0,:,:,:] = cp.array(([[0.5,0.5],[0.5,0.5],[0.5,0.5]],[[1,1],[1,1],[1,1]]))
                        r.reshape(1,2,3,2)
                        print(r)
                        

                        # theta shape: oxlxjxc
                        theta = (r * sigma + mu)
                        print("theta:{}".format(theta.shape))
                        # eta shape: oxlxc
                        # multiply all ftr_cols with given ftr_vector x
                        print(theta)
                        eta = cp.einsum("oljc,oj->oljc", theta, x_obs) 

                        # sum up all theta^cl_j * x_tj so we got l samples
                        # for all c classes
                        eta = cp.einsum("oljc->olc", eta) 
                        print("eta: {}".format(eta.shape))
                        print(eta)

                        # get a for numerical stability, shape oxl
                        a = cp.amax(eta, axis=2) * -1
                        print("a: {}".format(a.shape))
                        print(a)

                        eta = cp.einsum("olc->col", eta) + a
                        eta = cp.einsum("col->olc", eta)
                        print("eta with a: {}".format(eta.shape))
                        print(eta)


                        eta = cp.exp(eta) # we only need them exp
                        print("finally eta: {}".format(eta.shape))
                        print(eta)

                        # eta_sum shape: oxl
                        eta_sum = cp.einsum("olc->ol", eta)
                        print("sums:{}".format(eta_sum.shape))
                        print(eta_sum)
                        
                        # calculate softmax only for all classes
                        # divide all etas by eta_sum
                        softmax_all = np.einsum("olc,ol->olc", eta, (1/eta_sum))

                        print("softmax: {}".format(softmax_all.shape))
                        
                        # marginal shape: o
                        marginal = cp.einsum("ol->o",
                                             softmax_all[:,:,obs_class]) / \
                                   n_mc_samples

                        print("marginal: {}".format(marginal.shape))
                        print(marginal)


                        # calculate softmax derivative to theta
                        softmax_c = softmax_all[:,:,obs_class]
                        print("sm_c:{}".format(softmax_c.shape))

                        # first calculate derivative for all as k != c
                        softmax_derivative = -1 * cp.einsum("oj,ol,olc->oljc",
                                                            (x_obs),
                                                            softmax_c,
                                                            softmax_all)
                        print("derivavtive: {}".format(softmax_derivative.shape))
                        print(softmax_derivative)

                        # then for observed class c
                        softmax_derivative_c = cp.einsum("oj,ol,ol->olj",
                                                         x_obs,
                                                         softmax_c,
                                                         (1-softmax_c))
                        print("derivative c: {}".format(softmax_derivative_c.shape))
                        print(softmax_derivative_c)

                        softmax_derivative[:,:,:,obs_class] = \
                            softmax_derivative_c
                        
                        print("end derivative: {}".format(softmax_derivative.shape))
                        print(softmax_derivative)

                        nabla_mu = cp.einsum("oljc->ojc", softmax_derivative) /\
                                   n_mc_samples

                        print("nabla_mu: {}".format(nabla_mu.shape))
                        print(nabla_mu)

                        
                        
                        
                        nabla_sigma = cp.einsum("oljc,oljc->ojc",
                                                softmax_derivative,r) / \
                                      n_mc_samples
                        print("nablu_sigma: {}".format(nabla_sigma.shape))
                        print(nabla_sigma)
                                    

                        nabla_mu = cp.einsum("ojc->jco", nabla_mu)
                        mu += lr_mu * \
                                                cp.einsum("jco->jc",
                                                          (nabla_mu/ marginal))
                        print("mu:")
                        print(mu)

                        nabla_sigma = cp.einsum("ojc->jco", nabla_sigma)
                        sigma += lr_sigma * \
                                                   cp.einsum("jco->jc",
                                                             (nabla_sigma / 
                                                             marginal))
                        print("sigma:")
                        print(sigma)

                    except TypeError as e:
                            raise TypeError('All features must be a numeric data type.') from e

In [27]:
#test runs
__softmax(np.array((1,2,3)),1)

x:
[[1 2 3]]
y: 1
[[[[0.5 0.5]
   [0.5 0.5]
   [0.5 0.5]]

  [[1.  1. ]
   [1.  1. ]
   [1.  1. ]]]]
theta:(1, 2, 3, 2)
[[[[0.5 0.5]
   [0.5 0.5]
   [0.5 0.5]]

  [[1.  1. ]
   [1.  1. ]
   [1.  1. ]]]]
eta: (1, 2, 2)
[[[3. 3.]
  [6. 6.]]]
a: (1, 2)
[[-3. -6.]]
eta with a: (1, 2, 2)
[[[0. 0.]
  [0. 0.]]]
finally eta: (1, 2, 2)
[[[1. 1.]
  [1. 1.]]]
sums:(1, 2)
[[2. 2.]]
softmax: (1, 2, 2)
marginal: (1,)
[0.5]
sm_c:(1, 2)
derivavtive: (1, 2, 3, 2)
[[[[-0.25 -0.25]
   [-0.5  -0.5 ]
   [-0.75 -0.75]]

  [[-0.25 -0.25]
   [-0.5  -0.5 ]
   [-0.75 -0.75]]]]
derivative c: (1, 2, 3)
[[[0.25 0.5  0.75]
  [0.25 0.5  0.75]]]
end derivative: (1, 2, 3, 2)
[[[[-0.25  0.25]
   [-0.5   0.5 ]
   [-0.75  0.75]]

  [[-0.25  0.25]
   [-0.5   0.5 ]
   [-0.75  0.75]]]]
nabla_mu: (1, 3, 2)
[[[-0.25  0.25]
  [-0.5   0.5 ]
  [-0.75  0.75]]]
nablu_sigma: (1, 3, 2)
[[[-0.1875  0.1875]
  [-0.375   0.375 ]
  [-0.5625  0.5625]]]
mu:
[[-0.5  0.5]
 [-1.   1. ]
 [-1.5  1.5]]
sigma:
[[ 0.625  1.375]
 [ 0.25   1.75 ]
 [

In [19]:
sigma

array([[1., 1.],
       [1., 1.],
       [1., 1.]])

In [9]:
r.reshape(1,2,3,2)
r.shape

(2, 3, 2)

In [49]:
np.array([[ [-0.558, 1.555], [0.325, -0.726], [0.347,-0.159]], [[-0.955, 0.283], [0.115,-1.637], [-0.516,0.161]]])

array([[list([-0.558, 1.555]), list([0, 325, -0.726]),
        list([0.347, -0.159])],
       [list([-0.955, 0.283]), list([0.115, -1.637]),
        list([-0.516, 0.161])]], dtype=object)

In [ ]:
#for getting gaussnorm r ~ N(0,1) size samples x features
test = np.random.randn(10000,100)
print(test)
print(test[1])
print(test[9, 9])

In [ ]:
#how to calculate theta^cl_t
print(np.array([1,2,3,4,5]) * np.array([2,3,4,5,6]) + np.ones(5))

#for sampling create Matrix with all r monte_carlo X 


In [ ]:
# get theta at time t and for all featurs for class c is number of feature
theta_c = sigma[:,c] * np.random.randn(total_features) + mu[:,c]

In [38]:
bla = np.array([1,2,3,4])
np.sum(np.exp(bla))
bla2 = np.array((2,4,6))
np.einsum("i,j->ij", bla, bla2) * bla2

array([[ 2,  4,  6],
       [ 4,  8, 12],
       [ 6, 12, 18],
       [ 8, 16, 24]])

In [43]:
(np.einsum("j,l->jl", bla, bla2) * bla2) / bla2

array([[ 2.,  4.,  6.],
       [ 4.,  8., 12.],
       [ 6., 12., 18.],
       [ 8., 16., 24.]])

In [6]:
# tests for derivative o = 3, j = 4, l = 5 entries
X = np.array([[1,1,1,1], [2,2,2,2], [1,2,3,4]])
print(X)
eta = np.array([[1,2,3,4,5], [2,2,2,2,2], [3,3,3,3,3]])
sums_eta = np.array((5,5,5,5,5))
x_eta = np.einsum("oj,ol->ojl", X, eta)
print(x_eta)
final = np.einsum("ojl,ol->ojl", x_eta, (sums_eta-eta)) / sums_eta**2
x_eta[:,:,4]

[[1 1 1 1]
 [2 2 2 2]
 [1 2 3 4]]
[[[ 1  2  3  4  5]
  [ 1  2  3  4  5]
  [ 1  2  3  4  5]
  [ 1  2  3  4  5]]

 [[ 4  4  4  4  4]
  [ 4  4  4  4  4]
  [ 4  4  4  4  4]
  [ 4  4  4  4  4]]

 [[ 3  3  3  3  3]
  [ 6  6  6  6  6]
  [ 9  9  9  9  9]
  [12 12 12 12 12]]]


array([[ 5,  5,  5,  5],
       [ 4,  4,  4,  4],
       [ 3,  6,  9, 12]])

In [54]:
sums_eta - eta

array([[4, 3, 2, 1, 0],
       [3, 3, 3, 3, 3],
       [2, 2, 2, 2, 2]])

In [12]:
bla = x_eta[:,:,4].T
print(bla)
bla[:,2] += np.array((1,1,1,1))
bla

[[ 5  4  4]
 [ 5  4  7]
 [ 5  4 10]
 [ 5  4 13]]


array([[ 5,  4,  5],
       [ 5,  4,  8],
       [ 5,  4, 11],
       [ 5,  4, 14]])

In [ ]:
test = np.random.randn(2,3,4) #monte, features, classes
print(test.shape)
print("----------")
print(test[1])
print("-------")
print(test[1,1])
print("--------")
print(test[1,:,1])

In [3]:
test = np.array([[(1,2),(3,4)], [(5,6),(7,8)]])
ones = np.ones((2,2))
twos = np.ones((2,2)) * 2

In [4]:
test * twos + ones

array([[[ 3.,  5.],
        [ 7.,  9.]],

       [[11., 13.],
        [15., 17.]]])

In [4]:
test1 = test[0]
#print(test1)
#(test1.T * np.array([1,2])).T


In [9]:
c = np.einsum("ij,i->ij", test1, np.array((1,2)))
print(c)
c[0,1] #feature class

[[1 2]
 [6 8]]


2

In [5]:
test1

array([[1, 2],
       [3, 4]])

In [6]:
np.einsum("ij->j", test1) # for nabla

array([4, 6])

In [7]:
np.einsum("ij->i", test1)

array([3, 7])

In [101]:
test

array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]])

In [102]:
blub = np.einsum("ijk->ik", test)
print(blub) # -> blub[sample, class]
blub[0, 1]

[[ 4  6]
 [12 14]]


6

In [ ]:
a = np.arange(25).reshape(5,5)
b = np.arange(5)
a
print(test)
test2 = np.einsum("ijk,j->ijk", test, np.array((1,2)))


In [127]:
#test for softmax

def fun_test(x):
    print(x)
    result = np.empty(len(x))
    sum_all = np.sum(x)
    print(sum_all)
    for idx, entry in enumerate(x):
        result[idx] = entry/sum_all
    print("result: ", result)
    return result


fun_test_v = np.vectorize(fun_test)

test_array = np.array([[1,2],[3,4],[5,6]], dtype=np.float)

for idx, x in enumerate(test_array):
    test_array[idx] = fun_test(x)

test_array

[1. 2.]
3.0
result:  [0.33333333 0.66666667]
[3. 4.]
7.0
result:  [0.42857143 0.57142857]
[5. 6.]
11.0
result:  [0.45454545 0.54545455]


array([[0.33333333, 0.66666667],
       [0.42857143, 0.57142857],
       [0.45454545, 0.54545455]])

In [2]:
a = np.array((1,2,3,4,5,6))
a / a

array([1., 1., 1., 1., 1., 1.])

In [25]:
# test for simultaneous caclculations
test_matrix = np.array([[1,2,3,4,5],
                        [1,2,3,4,5],
                        [1,2,3,4,5],
                        [1,2,3,4,5]])
test_matrix[:,[0,1,0,2,3]]
test_matrix.T / np.array((1,2,3,4))
np.exp(test_matrix)

array([[  2.71828183,   7.3890561 ,  20.08553692,  54.59815003,
        148.4131591 ],
       [  2.71828183,   7.3890561 ,  20.08553692,  54.59815003,
        148.4131591 ],
       [  2.71828183,   7.3890561 ,  20.08553692,  54.59815003,
        148.4131591 ],
       [  2.71828183,   7.3890561 ,  20.08553692,  54.59815003,
        148.4131591 ]])

In [14]:
np.ones(4)

array([1., 1., 1., 1.])

In [35]:
for i in np.array((1)):
    print(i)

TypeError: iteration over a 0-d array

In [11]:
test = np.array((1))

In [8]:
type(test)

numpy.ndarray

In [10]:
test.shape == (2,)


True

In [14]:
for idx, x in np.nditer(test):
    print(idx, x)

TypeError: iteration over a 0-d array

In [46]:
np.ndarray((1), buffer=np.array([1]))

array([5.e-324])

In [58]:
# for sigma resizing
test = np.array([[-1,3],[5,-3]])

test[test<0] = 0
test

array([[0, 3],
       [5, 0]])

In [50]:
a, b = 1,2
print("{}, {}".format(a,b))

1, 2


In [53]:
len(test.shape)


2

In [55]:
len(np.array((1,2,3,4)).shape)

1

In [57]:
np.mean(test, axis=1)

array([1.5, 2.5])

In [61]:
np.array((1)).shape

()

In [60]:
len(np.array((1)))

TypeError: len() of unsized object

In [62]:
np.array((5)) + 6

11

In [70]:
for idx, x in enumerate(np.nditer(np.array((5,3,4,5)))):
    print(idx, x)

0 5
1 3
2 4
3 5


In [72]:
enumerate(np.nditer(3))

In [3]:
if True or \
False:
    print("bla")

bla


In [7]:
import numpy as np
probs = np.array((0.2, 0.3, 0.5))
mu = np.ones((3,3))
mu * probs

array([[0.2, 0.3, 0.5],
       [0.2, 0.3, 0.5],
       [0.2, 0.3, 0.5]])

In [10]:
np.sum(mu*probs, axis=1)

array([1., 1., 1.])